### 풀이
- (자력) 어떤 수 $A, B$를 합치는 것($AB$라고 표현)은 $A * 10^{|B|} + B$와 같다.
  - $\scriptsize |B|$는 $B$의 자릿수를 의미한다.
- $A$와 $B$를 합친것을 $AB$라고 할 때, $AB \mod k = (A * 10^{|B|} + B) \mod k = (A * 10^{|B|} \mod k + B \mod k) \mod k$이다.
  - 수식은 모듈러 연산의 분배법칙에 의해 유도된 것이다.
  - 따라서 숫자를 직접 합칠 필요 없이, 두 수의 나머지를 이용한 연산으로 나온 모든 값도, $k$가 100이하이므로, 모두 저장할 수 있다. (이 문제풀이의 핵심)
- `DP[i][j]` = 숫자를 합친 것이 `i` 상태일 때, 나머지가 j인 순열의 개수
  - `DP[현재 상태][합친 나머지] += DP[이전상태][이전 나머지]`

#### 순열에 대해
- DP의 상태를 나타내는 bit field가 (그것을 고르는)순열이 아닌 조합이 아닌가에 대한 의혹을 해결하기 위해 자세히 고찰해보았다.
- 예를들어 DP의 상태가 4bit라고 할 때, 0011은 0010과 0001에서 넘어온다.
  - 즉 0010에서 4번째 숫자를 추가한 경우와 0001에서 3번째 숫자를 추가한 경우와 같다고 볼 수 있다.
  - 따라서 0011은 이전 상태에서 순서를 고려했으므로 순열이라고 할 수 있다.
- 0111의 경우도 0011, 0101, 0110 에서 넘어오고, 이 과정에서 추가될 때 순서가 고려되므로 순열이라고 볼 수 있다.

In [ ]:
import math

N = int(input())
L = [int(input()) for _ in range(N)]
K = int(input())
DP = [[-1] * K for _ in range(1 << N)] # DP[bit][mod]
DP[0][0] = 1 # 0개를 골라서 나머지가 0인 경우는 1가지

def len(n) :
  return int(math.log10(int(n))) + 1

def p2(n) :
  return int(math.log2(n))

def sub(n):
  len = n.bit_length()
  for i in range(len):
    if n & (1 << i):
      sub = n & ~(1 << i)
      yield sub

def dp(bit, mod):
  if DP[bit][mod] != -1:
    return DP[bit][mod]

  DP[bit][mod] = 0
  for s in sub(bit): # s는 bit의 부분집합
    x = p2(bit^s)
    b = L[x]
    for j in range(K) :
      m = ((j * (10 ** len(b))) % K + b % K) % K
      DP[bit][m] += dp(s, j)
  return DP[bit][mod]
dp((1 << N) - 1, 0)

- 뭔가 이해를 잘못한 코드

In [ ]:
import sys
import math
input = sys.stdin.readline

n = int(input())
s = [int(input()) for _ in range(n)]
k = int(input())
r = [[(j*10**len(str(s[i]))+s[i])%k for j in range(k)] for i in range(n)]
dp = [[0]*k for _ in range(1<<n)]
dp[0][0]=1

for b in range(1<<n):
    for i in range(n):
        if b&(1<<i): continue
        for j in range(k):
            dp[b|(1<<i)][r[i][j]]+=dp[b][j]
p = dp[(1<<n)-1][0]
q = sum(dp[(1<<n)-1])
g = math.gcd(p,q)
print(f'{p//g}/{q//g}')